In [46]:
import requests
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import datetime as dt

# Definimos path de chromedriver
PATH = 'D:\Program Files\chromedriver.exe'
driver = webdriver.Chrome(PATH)

# Definimos credenciales - SE TIENE QUE HACER MANUALMENTE
userid = 'USUARIO'
password = 'PASSWORD'

# Definimos entidades a buscar a partir de scraping de ENISA
entity_list = pd.read_csv('df_enisa.csv', sep=';')
entity_list = entity_list["Marca"].to_list()
# Empezamos con 10 entidades
entity_list = entity_list[0:10]

# Abrimos el primer driver con la página de entrada de Linkedin
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

# Hacemos login
user = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "username"))    )
user.send_keys(userid)
pwd = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "password"))    )
pwd.send_keys(password)
pwd.submit()

# Definimos variables
data = {}
header = ["Marca", "Sitio web", "Sector", "Tamaño de la empresa","Sede", "Tipo"] 
df_linkedin = pd.DataFrame(columns=header)

# Empezamos a iterar sobre el listado de empresas
for entity in entity_list:
    data["Marca"] = entity
    # Buscaremos la compañia en la barra de búsqueda superior
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "main")))
    companyID = driver.find_element_by_class_name('search-global-typeahead__input')
    companyID.clear()
    companyID.send_keys(entity)
    time.sleep(1)
    companyID.send_keys(Keys.ARROW_DOWN)
    companyID.send_keys(Keys.RETURN)
    try:
        page_loaded = WebDriverWait(driver, 5).until(
            EC.title_contains("Resumen"))
        home = driver.current_url
        about = home + 'about'
        # Accedemos a la página de "Acerca de" de la empresa
        driver.get(about)
        page_loaded = WebDriverWait(driver, 10).until(
            EC.title_contains("Acerca"))
        
        # Pasamos a scrapear la página con bs4 - Recuperar datos útiles
        bs_about = driver.page_source
        soup = BeautifulSoup(bs_about, 'lxml')
        item_list = soup.find('dl')
        for item in item_list.find_all('dt'):
            info = str.strip(item.text)
            # print(info)
            value = str.strip(item.find_next_sibling("dd").text)
            if (info in (header)): data[info]=value
            # print(value)
        df_linkedin=df_linkedin.append(data, ignore_index=True)
    except:
        # Empresa NA - Pasamos a la siguiente
        pass

driver.quit()

# Creamos el CSV con los datos
df_linkedin.to_csv('df_linkedin.csv', sep=";", na_rep= "",)


Hora inicio proceso: 2021-03-29 00:19:13.057041
